In [85]:
import subprocess
import os
from PIL import Image, ImageOps
import pytesseract
full_img_path = r"cropped/cropped_screenshot.png"
button_img_path = r"cropped/cropped_button.png"

pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"

def take_android_screenshot(save_path):
    try:
        subprocess.run(["adb", "start-server"], check=True)
    except subprocess.CalledProcessError:
        print("ADB is not installed or not added to the PATH.")
        return

    device_path = "/sdcard/screenshot.png"
    local_path = os.path.join(save_path, "screenshot.png")

    try:
        subprocess.run(["adb", "shell", f"screencap -p {device_path}"], check=True)
        print("Screenshot taken successfully.")
    except subprocess.CalledProcessError:
        print("Failed to take screenshot on device.")
        return

    try:
        subprocess.run(["adb", "pull", device_path, local_path], check=True)
        print(f"Screenshot saved to {local_path}")
    except subprocess.CalledProcessError:
        print("Failed to pull the screenshot from device.")
        return
    

def crop_image_full(image_path, save_path):
    try:
        with Image.open(image_path) as img:
            # Define the area to crop (left, top, right, bottom)
            crop_area = (100, 900, 967, 1050)  # Adjust this as needed
            cropped_img = img.crop(crop_area)
            cropped_path = os.path.join(save_path, "cropped_screenshot.png")
            cropped_img.save(cropped_path)
            print(f"Cropped image saved to {cropped_path}")
    except Exception as e:
        print(f"Failed to crop the image: {str(e)}")

def crop_image_full_button(image_path, save_path):
    try:
        with Image.open(image_path) as img:
            # Define the area to crop (left, top, right, bottom)
            crop_area = (747, 934, 932, 1020)  # Adjust this as needed
            cropped_img = img.crop(crop_area)
            cropped_path = os.path.join(save_path, "cropped_button.png")
            cropped_img.save(cropped_path)
            print(f"Cropped image saved to {cropped_path}")
    except Exception as e:
        print(f"Failed to crop the image: {str(e)}")

def read_text_from_image(image_path):
    try:
        # Open an image file
        with Image.open(image_path) as img:
            config = '--oem 3 --psm 6 outputbase digits'
            text = pytesseract.image_to_string(img, config=config)
            return text.strip().split('\n')
    except Exception as e:
        print(f"Failed to process the image: {str(e)}")
        return None
    

directory = r"F:/python/Eatventure OCR/images"
#take_android_screenshot(directory)
#crop_image_full("images\screenshot.png","cropped")
#crop_image_full_button("images\screenshot.png","cropped")
#read_text_from_image("cropped\cropped_screenshot.png")
# print(read_text_from_image(full_img_path)[0])
# print(read_text_from_image(full_img_path)[1])
print(read_text_from_image(r"cropped/3.png"))


['500']
